In [0]:
%pip install -r requirements_base.txt

%restart_python

In [0]:
%load_ext autoreload
%autoreload 2 

In [0]:
#url = "https://docs.google.com/spreadsheets/d/1PK2eEqNqX276YBgOOWz7MHWpWXAyKo5J1Bil7BgmsCs/edit?gid=0#gid=0"

#languages = "Spanish (Spain), German"

In [0]:
import pandas as pd
import json
import random
import openai
from openai import OpenAI
from pathlib import Path
from unidecode import unidecode
import gspread
import os


import pyspark
from pyspark.sql.functions import *
from functools import reduce
from typing import *

import time
import datetime
import re
import tiktoken

from bert_score import score as bert_score


import mlflow

In [0]:
%run "./setup"

In [0]:
#test_json = "{"Timestamp": "6/23/2025 9:42:44", "SubmitterEmail": "vanessa.mendoza@jamcity.com", "DueDate": "7/11/2025", "Game": "Harry Potter: Hogwarts Mystery", "TargetLanguages": "Spanish (Latin America), French (France), German, Korean, Italian, Japanese, Simplified Chinese, Traditional Chinese (Taiwan), Portuguese (Brazil)", "URL": "https://docs.google.com/spreadsheets/d/1T-bKc0iLwmnsUYRhv3nGoDBkJyBdOi8euyaDuHyYTtE/edit?usp=sharing", "QAFlag": "Yes", "ShareStatus": "Shared", "SyncedAt": "2025-07-03T20:24:22.704Z", "PermissionStatus": "", "DataValid": "", "InternalStatus": "", "LongResultSheetURL": "", "FormattedResultSheetURL": "", "ProofreadSheetURL": "", "ProofreadCompletedSheetURL": "", "JobStatus": "NotStarted", "RowFingerprint": "1XkcFnA9LvoEe8xt64Edtg==", "JobCompletedTimestamp": "", "JobRetryFlag": "", "SourceCD": "ASO Monthly"}"

In [0]:
job_dict = handle_json_parsing()

In [0]:
%run "./authenticationScript"

In [0]:
def get_aso_data(job_dict, platform, sh): 

    def get_values_from_sheet(sh, platform):
        if platform == 'ios':
            return sh.worksheet(platform).get_all_values()
        if platform == 'android':
            return sh.worksheet(platform).get_all_values()
    # Helper 
    def check_values_get_inputs(sh, platform, job_dict):

        values = get_values_from_sheet(sh, platform)
        if len(values)<4:
            return None
        
        headers, data = values[0:3],values[3:]
        if platform=='ios':
            df = pd.DataFrame(data,columns = ['en_US_30','en_US_50','en_US_120'])
        if platform=='android':
            df = pd.DataFrame(data, columns = ['en_US_80','en_US_500'])

        df['row_id'] = df.index
        df['RowFingerprint'] = job_dict['row_fingerprint']
        df['platform'] = platform
        df['game'] = job_dict['game_name']  
        return df
    
    df = check_values_get_inputs(sh, platform,job_dict)
    if df is not None:
            return df

    return None


def get_data(job_dict):

    gc = get_gspread_client_from_secret()
    try:
        sheet = gc.open_by_url(job_dict['workbook_url'])
    except:
        raise Exception("Cannot open workbook url!")
    
    if job_dict['source_cd'] == "ASO Monthly":
        print('getting aso monthly data')
        ios_df = get_aso_data(job_dict, 'ios', sheet) 
        android_df = get_aso_data(job_dict, 'android', sheet)
        # Get long data from both input sheets
        df = process_aso_to_long(ios_df, android_df)
        return df

    if job_dict['source_cd']=="ASO Ad-Hoc":
        print('getting aso adhoc data')
        #df = get_aso_adhoc(sheet)
        return None
    
    if job_dict['source_cd'] == "PP" or job_dict['source_cd'] == "CS":
        print('getting pp or cs data')
        wksht = sheet.worksheet("input")
        vals = source_wksht.get_all_values()
        headers, data = vals[0], vals[1:]
        df = pd.DataFrame(data, columns=headers)
        return df
    
    return None

In [0]:
df = get_data(job_dict)

In [0]:
df.display()

In [0]:
ios_inputs

In [0]:
process_aso_to_long(ios_inputs, android_inputs)

In [0]:
def process_aso_to_long(ios_inputs, android_inputs): 
 
    q_ios = f"""
            select 
                game,
                platform,
                RowFingerprint,
                row_id,
                'title' as type_desc,
                en_US_30 as en_US,
                30 as en_char_limit
            from df
            union all
            select 
                game,
                platform,
                RowFingerprint,
                row_id,
                'short_description' as type_desc,
                en_US_50 as en_US,
                50 as en_char_limit
            from df
            union all
            select 
                game,
                platform,
                RowFingerprint,
                row_id,
                'long_description' as type_desc,
                en_US_120 as en_US,
                120 as en_char_limit
            from df
    """


    #TODO: TEST THIS
    q_android= f""" select 
                game,
                platform,
                RowFingerprint,
                row_id,
                'short_description' as type_desc,
                en_US_80 as en_US,
                80 as en_char_limit
            from df
            union all
            select 
                game,
                platform,
                RowFingerprint,
                row_id,
                'long_description' as type_desc,
                en_US_500 as en_US,
                500 as en_char_limit
            from df
        """

    dfs_to_unions = []

    if ios_inputs is not None:
        spark.createDataFrame(ios_inputs).createOrReplaceTempView('df')
        long_ios = spark.sql(q_ios).toPandas()
        dfs_to_unions.append(long_ios)


    if android_inputs is not None:
        spark.createDataFrame(android_inputs).createOrReplaceTempView('df')
        long_android = spark.sql(q_android).toPandas()
        dfs_to_unions.append(long_android)

        #pdf = pd.concat([])
    pdf = pd.concat(dfs_to_unions) 
    
    return pdf

In [0]:
dfs_to_unions = []

if ios_inputs is not None:
    spark.createDataFrame(ios_inputs).createOrReplaceTempView('df')
    long_ios = spark.sql(q_ios).toPandas()
    dfs_to_unions.append(long_ios)


if android_inputs is not None:
    spark.createDataFrame(android_inputs).createOrReplaceTempView('df')
    long_android = spark.sql(q_android).toPandas()
    dfs_to_unions.append(long_android)

    #pdf = pd.concat([])
pdf = pd.concat(dfs_to_unions)    


In [0]:
pdf.display()

In [0]:
long_ios.display()

In [0]:
def get_aso_data(job_dict, platform, sh): 

    def get_values_from_sheet(sh, platform):
        if platform == 'ios':
            return sh.worksheet(platform).get_all_values()
        if platform == 'android':
            return sh.worksheet(platform).get_all_values()
    # Helper 
    def check_values_get_inputs(sh, platform, job_dict):

        values = get_values_from_sheet(sh, platform)
        if len(values)<4:
            return None
        
        headers, data = values[0:3],values[3:]
        if platform=='ios':
            df = pd.DataFrame(data,columns = ['en_US_30','en_US_50','en_US_120'])
        if platform=='android':
            df = pd.DataFrame(data, columns = ['en_US_80','en_US_500'])

        df['row_id'] = df.index
        df['RowFingerprint'] = job_dict['row_fingerprint']
        df['platform'] = platform
        df['game'] = job_dict['game_name']  
        return df
    
    df = check_values_get_inputs(sh, platform,job_dict)
    if df is not None:
            return df

    return None

In [0]:
def process_to_long(df, platform, job_dict):

    df = spark.createDataFrame(df)

    def helper_by_platform_language(df, platform, language,job_dict):
        lang_cd = job_dict['all_language_map'][language]
        if platform == 'android' and lang_cd in ['ja_JP','ko_KR','zh_CN','zh_TW']:
            altered = df.withColumn("target_char_limit", 
                                (col('en_char_limit')/2).cast(IntegerType()))
        else:
            altered = df.withColumn("target_char_limit", col('en_char_limit'))
            
        altered = altered.withColumn("language_cd", 
                                    lit(lang_cd))\
                        .withColumn('language',lit(language))\
                        .withColumn(
                                "row_idx",
                                concat_ws(
                                    "::",
                                    concat_ws("_", lit("row"), col("row_id").cast("string")),
                                    col("game"),
                                    col("platform"),
                                    col("language_cd"),
                                    col("en_char_limit").cast("string")
                                )
                            )
        return altered.toPandas()
    
    df_holder = []
    for lang in job_dict['all_languages']:
        lang_df = helper_by_platform_language(df, platform, lang,job_dict)
        df_holder.append(df_holder)

    return pd.concat(df_holder)



In [0]:
ios_inputs, android = get_data(job_dict)

In [0]:
job_dict['all_language_map']

In [0]:
ios_inputs

In [0]:
process_to_long(ios_inputs,'ios',job_dict)

In [0]:
get_aso_data(job_dict,'android', source_ss)

In [0]:
get_aso_data(job_dict,'ios', source_ss)

In [0]:
ios, android = get_data(job_dict)

In [0]:
#

In [0]:
ios

In [0]:
#basically... get_aso_data()

#url=job_dict['workbook_url']
#ss =gc.open_by_url(url)
#ss.worksheet('ios').get_all_values()

In [0]:
job_dict

In [0]:
#get job dict for ASO Monthly
gc = get_gspread_client_from_secret()


source_ss = gc.open_by_url("https://docs.google.com/spreadsheets/d/1WnCBGk1V1nEUeU7aPWBqrQXczmaMop6AbGmkDKkAQcE/edit?gid=1033119277#gid=1033119277")

#

source_wksht = source_ss.worksheet('Validation and Url Tracking')
vals = source_wksht.get_all_values()


headers, data = vals[0],vals[1:]
df = pd.DataFrame(data, columns=headers)

In [0]:
to_process_df = df[df.JobStatus == 'NotStarted']


row_dict = to_process_df.iloc[0].to_dict()
row_dict['SourceCD'] = 'ASO Monthly'

JsonJobSlug = json.dumps(row_dict)


JsonJobSlug


#for every json row dict
#### So would then pass this to dbutils.notebook.run("", {'JsonJobSlug':JsonJobSlug})

In [0]:
job_dict

In [0]:
#PP Specific validation

def validate_sheet_wksht_format_return_df(workbook_url):
    try:
       ss = gc.open_by_url(workbook_url)
    except:
        raise Exception(f'Unable to open workbook {workbook_url}')

    worksheets = ss.worksheets()
    worksheet_titles = [x.title for x in worksheets]
    if 'input' not in worksheet_titles:
        raise Exception(f"Unable to find 'input' worksheet in workbook {workbook_url}")
    else:
        input_wksht = ss.worksheet('input')
        ### Validate data
        input_values = input_wksht.get_all_values()
        in_header, in_data = input_values[0], input_values[1:]
        
        if len(in_header) != 3:
            raise Exception(f"Invalid number of columns in input worksheet. Expected 3, found {len(in_header)}")

        expected_cols = ['token', 'context', 'en_US']
        for col in in_header:
            if col not in expected_cols:
                raise Exception(f"Invalid column name '{col}' in input worksheet. Expecting columns['token','context','en_US']")
        #print("Num rows in input data:", len(in_data))

        if len(in_data) == 0:
            raise Exception(f"Invalid number of rows in input worksheet. Expected at least 1, found 0")

        init_df = spark.createDataFrame(in_data, ['token','context','en_US'])

    if 'output' not in worksheet_titles:
        print("Not finding 'output' worksheet in workbook {workbook_url}...")
        print("creating 'output' worksheet")
        ss.add_worksheet('output', 400, 50)
        output_wksht = ss.worksheet('output')
        output_wksht.update([['token', 'en_US', *cds]],"A1:Q1")
    if 'summary_stats' not in worksheet_titles:
        print("Not finding 'summary_stats' worksheet in workbook {workbook_url}...")
        print("creating 'summary_stats' worksheet")
        ss.add_worksheet('summary_stats', 50, 50)
        #header_line = ['Metric', *cds]
        summary_stats_sheet = ss.worksheet('summary_stats')
        summary_stats_sheet.update([['Metric', *cds]],"A1:P1")

    return init_df.toPandas()

def get_data_slug(df):
    data_to_print = df.to_dict('records')
    slug = json.dumps(data_to_print, ensure_ascii=False, separators=(",", ":"))
    return slug

In [0]:
%run "./authenticationScript"

In [0]:
gc = get_gspread_client_from_secret()

In [0]:
from config import *
#from language import *
#from translation import *
#from scoring import *

In [0]:
def get_data(job_dict:dict)->pd.DataFrame:

    source_cd = job_dict['source_cd']
    if source_cd == "PP":
       #TODO: prob import this function from a specific PP module
       data = validate_sheet_wksht_format_return_df(job_dict['workbook_url'])
       data_slug = get_data_slug(data)
        
#   if source_cd == "ASO Ad-Hoc":
#        #data = ..
#        #data_slug = ..
#  
#   #if source_cd == "ASO Monthly":
#       #data = ..
#       #data_slug=..
#   
#   #if source_cd =="CS":
#       #data = ...
#       #data_slug = ...


    job_dict['inputs_df'] = data
    job_dict['data_slug']= data_slug

    return job_dict

In [0]:
job_dict = get_data(job_dict)

In [0]:
job_dict

In [0]:
## ASO will be different... 



# CS - will need to get ['network','character_limit','en_US']

In [0]:
# Get Data for PP
#df = validate_sheet_wksht_format_return_df(job_dict['workbook_url'])
# And the Input Slug for PP
#data_slug = get_data_slug(df)

In [0]:
for lang,lang_cd in job_dict['all_language_map'].items():
    print(f"lang: {lang},lang_cd: {lang_cd}")
    #get_PP_prompt(lang, job_dict['data_slug'])

In [0]:
####

In [0]:
# Preproces for ASO Monthly

In [0]:
#Preprocess for ASO AdHoc

In [0]:
#Preprocess for CS

In [0]:
#Preprocess for Other




